In [3]:
import pandas as pd
dataset_path = "/kaggle/input/media-coverage-speeches-unmerged"
print(os.listdir(dataset_path))
news = pd.read_csv(f"{dataset_path}/full_mediacoverage_preprocessed.csv")
speeches = pd.read_csv(f"{dataset_path}/full_data_updated_24.csv")

# Display first few rows
print("News Data:")
print(news.head())

print("\nSpeeches Data:")
print(speeches.head())


['full_mediacoverage_preprocessed.csv', 'full_data_updated_24.csv']
News Data:
          Index                                               Link  \
0  1_01_12_2018  https://www.bbc.com/mundo/noticias-america-lat...   
1  2_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
2  3_01_12_2018  https://oem.com.mx/elsoldemexico/mexico/en-don...   
3  4_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
4  5_01_12_2018  https://www.eleconomista.com.mx/politica/Nicol...   

                                              Domain  \
0  BBC\nToma de protesta de AMLO: las 5 tradicion...   
1  Expansión Política\nAMLO rinde protesta y prom...   
2  El Sol de México\n¿Hay Ley Seca este 1 de dici...   
3  Expansión Política\nAMLO es un "líder persiste...   
4  El Economista\nNicolás Maduro llega a Palacio ...   

                                               Title        Date  \
0  Toma de protesta de AMLO: las 5 tradiciones qu...  2018-12-01   
1        AMLO rinde protest

In [6]:
print(news.columns)
print(news.shape)
print(speeches.columns)
print(speeches.shape)

Index(['Index', 'Link', 'Domain', 'Title', 'Date', 'Content', 'month_abbr',
       'processed_content'],
      dtype='object')
(36563, 8)
Index(['Unnamed: 0', 'X', 'speech_id', 'title', 'urls', 'content', 'date'], dtype='object')
(2363, 7)


In [5]:
import os
os.listdir('/kaggle/input')

['sentence-transformer-esp', 'media-coverage-speeches-unmerged']

In [7]:
import pandas as pd

def chunk_text_by_sentences(text, chunk_size=4):
    """Splits text into smaller chunks of `chunk_size` sentences."""
    if pd.isna(text):  # Handle missing values
        return []
    sentences = text.split('. ')  # Basic sentence split
    return ['. '.join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]


In [8]:
# ✅ Apply chunking function
news["news_chunks"] = news["processed_content"].apply(chunk_text_by_sentences)
speeches["speech_chunks"] = speeches["content"].apply(chunk_text_by_sentences)


In [9]:
# ✅ Explode DataFrames (each chunk = new row)
news = news.explode("news_chunks").reset_index(drop=True)
speeches = speeches.explode("speech_chunks").reset_index(drop=True)
print("News Data after chunking:")
print(news.head())

print("\nSpeeches Data after chunking:")
print(speeches.head())

# Check dataset shapes
print("News dataset shape:", news.shape)
print("Speeches dataset shape:", speeches.shape)


News Data after chunking:
          Index                                               Link  \
0  1_01_12_2018  https://www.bbc.com/mundo/noticias-america-lat...   
1  2_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
2  3_01_12_2018  https://oem.com.mx/elsoldemexico/mexico/en-don...   
3  4_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
4  5_01_12_2018  https://www.eleconomista.com.mx/politica/Nicol...   

                                              Domain  \
0  BBC\nToma de protesta de AMLO: las 5 tradicion...   
1  Expansión Política\nAMLO rinde protesta y prom...   
2  El Sol de México\n¿Hay Ley Seca este 1 de dici...   
3  Expansión Política\nAMLO es un "líder persiste...   
4  El Economista\nNicolás Maduro llega a Palacio ...   

                                               Title        Date  \
0  Toma de protesta de AMLO: las 5 tradiciones qu...  2018-12-01   
1        AMLO rinde protesta y promete no reelegirse  2018-12-01   
2  ¿Hay Ley 

In [1]:
import os
print(os.listdir("/kaggle/input/sentence-transformer-esp"))
import os

model_path = "/kaggle/input/sentence-transformer-esp/sentence_similarity_spanish_es"
print(os.listdir(model_path))

['sentence_similarity_spanish_es']
['config.json', '1_Pooling', 'similarity_evaluation_stsb-multi-mt-test_results.csv', 'README.md', 'tokenizer.json', 'tokenizer_config.json', 'sentence_bert_config.json', 'pytorch_model.bin', 'eval', 'config_sentence_transformers.json', 'model.safetensors', 'modules.json', 'special_tokens_map.json', '.gitattributes', '.git', 'vocab.txt']


In [11]:
from sentence_transformers import SentenceTransformer

# Set the model path based on your uploaded folder
model_path = "/kaggle/input/sentence-transformer-esp/sentence_similarity_spanish_es"

# Load the model using the path
model = SentenceTransformer(model_path, device="cuda")  # Ensure GPU usage
print("Model loaded successfully!")


Model loaded successfully!


In [12]:
import numpy as np
import torch
from tqdm import tqdm

def generate_embedding_batch(text_list, batch_size=128, name="Dataset"):
    """Generate embeddings in batches to optimize memory."""
    embeddings = []
    for i in tqdm(range(0, len(text_list), batch_size), desc=f"Generating {name} Embeddings", unit="batch"):
        batch = text_list[i:i+batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True, device="cuda").cpu().numpy()
        embeddings.append(batch_embeddings)

        # Print progress every 1,000 batches
        if (i // batch_size) % 1000 == 0:
            print(f"Processed {i} chunks for {name}")

    return np.vstack(embeddings)  # Stack into a single NumPy array


In [ ]:
news["news_embeddings"] = list(generate_embedding_batch(news["processed_content"].tolist(), name="News"))
print("News embeddings generated successfully!")
news.to_csv("news_with_embeddings.csv", index=False)


In [ ]:
news.to_csv('/kaggle/working/news_with_embeddings.csv', index=False)

from IPython.display import FileLink
FileLink(r'news_with_embeddings.csv')

In [14]:
speeches["speech_embeddings"] = list(generate_embedding_batch(speeches["content"].tolist(), name="Speeches"))
print("Speech embeddings generated successfully!")

Generating Speeches Embeddings:   0%|          | 0/1366 [00:00<?, ?batch/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generating Speeches Embeddings:   0%|          | 1/1366 [00:04<1:40:56,  4.44s/batch]

Processed 0 chunks for Speeches


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generating Speeches Embeddings: 100%|█████████▉| 1365/1366 [1:17:44<00:03,  3.81s/batch]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generating Speeches Embeddings: 100%|██████████| 1366/1366 [1:17:47<00:00,  3.42s/batch]


Speech embeddings generated successfully!


In [15]:
speeches.to_csv("speeches_with_embeddings.csv", index=False)
from IPython.display import FileLink
FileLink(r'speeches_with_embeddings.csv')

/kaggle/working/speeches_with_embeddings.csv

In [1]:
# Check if speech_embeddings exist and their shape
print(subset_df['speech_embeddings'].head())

NameError: name 'subset_df' is not defined

# Coding similarity algorithm

In [2]:
import os
os.listdir('/kaggle/input')
embeddings_path = "/kaggle/input/embeddings"
print(os.listdir(embeddings_path))

['speeches_with_embeddings.csv', 'news_with_embeddings.csv']


In [3]:
import pandas as pd

speeches_embeddings = pd.read_csv(f"{embeddings_path}/speeches_with_embeddings.csv")
news_embeddings = pd.read_csv(f"{embeddings_path}/news_with_embeddings.csv")

# Display first few rows
print("News Data:")
print(news_embeddings.head())

print("\nSpeeches Data:")
print(speeches_embeddings.head())


News Data:
          Index                                               Link  \
0  1_01_12_2018  https://www.bbc.com/mundo/noticias-america-lat...   
1  2_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
2  3_01_12_2018  https://oem.com.mx/elsoldemexico/mexico/en-don...   
3  4_01_12_2018  https://politica.expansion.mx/presidencia/2018...   
4  5_01_12_2018  https://www.eleconomista.com.mx/politica/Nicol...   

                                              Domain  \
0  BBC\nToma de protesta de AMLO: las 5 tradicion...   
1  Expansión Política\nAMLO rinde protesta y prom...   
2  El Sol de México\n¿Hay Ley Seca este 1 de dici...   
3  Expansión Política\nAMLO es un "líder persiste...   
4  El Economista\nNicolás Maduro llega a Palacio ...   

                                               Title        Date  \
0  Toma de protesta de AMLO: las 5 tradiciones qu...  2018-12-01   
1        AMLO rinde protesta y promete no reelegirse  2018-12-01   
2  ¿Hay Ley Seca este 1 de 

In [5]:
print(speeches_embeddings.columns)

Index(['Unnamed: 0', 'X', 'speech_id', 'title', 'urls', 'content', 'date',
       'speech_chunks', 'speech_embeddings'],
      dtype='object')


In [6]:
print(type(news_embeddings['news_embeddings'].iloc[0]))
#since it is a class 'str' then we need to convert the embedding as a numpy array

<class 'str'>


In [7]:
# Convert 'date' column to datetime if it is not already in datetime format
speeches_embeddings['date'] = pd.to_datetime(speeches_embeddings['date'], errors='coerce')
news_embeddings['Date'] = pd.to_datetime(news_embeddings['Date'], errors='coerce')

# Subset the data to 2019 and reset indices

news_2019_reset = news_embeddings[news_embeddings['Date'].dt.year == 2019].reset_index(drop=True)
speeches_2019_reset = speeches_embeddings[speeches_embeddings['date'].dt.year == 2019].reset_index(drop=True)

# Now you can proceed with the cleaned data


In [8]:
import numpy as np

# Function to clean and convert the embeddings
def clean_and_convert(embedding_str):
    # Remove any unwanted characters like brackets or newline characters
    cleaned_str = embedding_str.replace('[', '').replace(']', '').replace('\n', '')
    
    # Convert the cleaned string to a numpy array
    return np.array(cleaned_str.split()).astype(float)

In [9]:
import torch 

# Step 2: Clean and convert the embeddings for 2019 data
cleaned_news_embeddings_2019 = news_2019_reset['news_embeddings'].apply(clean_and_convert)
cleaned_speech_embeddings_2019 = speeches_2019_reset['speech_embeddings'].apply(clean_and_convert)

# Verify the cleaned embeddings
print("Cleaned news embedding (first sample) for 2019:", cleaned_news_embeddings_2019.iloc[0])
print("Cleaned speech embedding (first sample) for 2019:", cleaned_speech_embeddings_2019.iloc[0])


Cleaned news embedding (first sample) for 2019: [ 7.00545609e-02  1.59867704e-01  1.82033628e-01 -1.10902913e-01
 -1.68071046e-01  7.82872558e-01  1.24405897e+00  9.35690045e-01
 -8.79230499e-01  1.69901311e-01  3.14941794e-01 -8.36316347e-01
  1.94153115e-01 -1.05180137e-01 -1.43057257e-01 -4.09085423e-01
  4.37348098e-01  3.12500400e-03  5.30271530e-01 -4.83261496e-02
 -7.44194686e-01  5.34093797e-01  6.49602234e-01 -1.16227746e+00
 -8.81177306e-01 -6.07623994e-01  7.66389966e-01  3.55166703e-01
 -4.08001542e-01  5.95054999e-02 -4.70891893e-01 -3.36702853e-01
  2.59495556e-01 -8.38763833e-01 -2.42981687e-01 -3.25389415e-01
 -3.09586227e-01  3.96134198e-01  4.72463161e-01 -6.93257987e-01
 -3.02345544e-01 -5.06430805e-01  3.93638819e-01  8.52328613e-02
  5.45643926e-01  6.85154796e-01 -1.47084430e-01  2.22051278e-01
 -5.23011625e-01 -5.16374409e-01  1.38684058e+00 -5.68437874e-01
 -4.49357063e-01 -1.91186011e-01 -2.05529436e-01 -5.80088556e-01
 -2.78938144e-01  3.71591657e-01  1.321355

In [10]:
import torch
import numpy as np

news_matrix_2019 = np.vstack(cleaned_news_embeddings_2019.values)
speech_matrix_2019 = np.vstack(cleaned_speech_embeddings_2019.values)


# Convert NumPy arrays to PyTorch tensors and move to GPU
news_tensor_2019 = torch.tensor(news_matrix_2019, dtype=torch.float32).cuda()
speech_tensor_2019 = torch.tensor(speech_matrix_2019, dtype=torch.float32).cuda()


In [ ]:
import time
import torch
import pandas as pd

# Function to compute batch cosine similarity
def compute_batch_cosine_similarity(embedding, batch_embeddings):
    embedding = embedding.view(1, -1)  # Reshape to (1, embedding_size)
    embedding = embedding / embedding.norm(dim=1, keepdim=True)  # Normalize the speech embedding
    batch_embeddings = batch_embeddings / batch_embeddings.norm(dim=1, keepdim=True)  # Normalize news embeddings
    
    # Compute cosine similarity for the batch
    return torch.mm(batch_embeddings, embedding.mT).squeeze()  # Use mT for matrix transpose

# Prepare an empty list to store the results
similarities_2019 = []

# Track the start time
start_time = time.time()
iteration_count = 0
progress_report_every = 1000

# Compute similarity for all speeches and news in 2019
for idx, speech_data in speeches_2019_reset.iterrows():
    speech_embedding_2019 = speech_tensor_2019[idx]  # Access the tensor by index
    speech_date_2019 = speech_data['date']
    
    # Compute cosine similarity for all news embeddings at once
    cosine_similarities = compute_batch_cosine_similarity(speech_embedding_2019, news_tensor_2019)
    
    # Store results
    for news_id, news_data in news_2019_reset.iterrows():
        news_date_2019 = news_data['Date']
        similarities_2019.append({
            'speech_id': idx,
            'news_id': news_id,
            'speech_date': speech_date_2019,
            'news_date': news_date_2019,
            'cosine_similarity': cosine_similarities[news_id].item()
        })
    
    # Increment iteration count
    iteration_count += 1

    # Print progress every 1,000 iterations
    if iteration_count % progress_report_every == 0:
        elapsed_time = time.time() - start_time
        print(f"Processed {iteration_count}/{len(speeches_2019_reset)} speeches. Time elapsed: {elapsed_time:.2f}s")

# Convert similarities to a DataFrame for analysis
similarities_df_2019 = pd.DataFrame(similarities_2019)
print("Finished computing cosine similarities.")


Processed 1000/32874 speeches. Time elapsed: 286.73s
Processed 2000/32874 speeches. Time elapsed: 574.72s
Processed 3000/32874 speeches. Time elapsed: 860.18s
Processed 4000/32874 speeches. Time elapsed: 1158.43s
Processed 5000/32874 speeches. Time elapsed: 1458.32s
Processed 6000/32874 speeches. Time elapsed: 1755.62s


In [11]:
import time
import torch

# Function to compute cosine similarity between two embeddings
def compute_cosine_similarity(embedding1, embedding2):
    # Reshape embeddings to 2D tensors with shape (1, embedding_size)
    embedding1 = embedding1.view(1, -1)  # Ensure it's a 2D tensor (1, embedding_size)
    embedding2 = embedding2.view(1, -1)  # Ensure it's a 2D tensor (1, embedding_size)
    
    # Calculate cosine similarity using the dot product and norms
    return torch.mm(embedding1, embedding2.T).cpu().numpy()[0][0]

# Prepare an empty list to store the results
similarities_2019 = []

# Track the start time to estimate processing duration
start_time = time.time()

# Compute similarity for all speeches and news in 2019
for idx, speech_data in speeches_2019_reset.iterrows():
    speech_embedding_2019 = speech_tensor_2019[idx]  # Use the index of the speech DataFrame to access tensor
    speech_date_2019 = speech_data['date']
    
    for news_id, news_data in news_2019_reset.iterrows():
        news_embedding_2019 = news_tensor_2019[news_id]  # Use the index of the news DataFrame to access tensor
        news_date_2019 = news_data['Date']
        
        # Compute cosine similarity between the embeddings
        cosine_sim = compute_cosine_similarity(speech_embedding_2019, news_embedding_2019)
        
        # Append the result to the list
        similarities_2019.append({
            'speech_id': idx,  # Use the index from speeches_2019
            'news_id': news_id,
            'speech_date': speech_date_2019,
            'news_date': news_date_2019,
            'cosine_similarity': cosine_sim
        })
    
    # Track the elapsed time after processing each speech
    elapsed_time = time.time() - start_time
    print(f"Processed speech {idx + 1}/{len(speeches_2019_reset)}. Time elapsed: {elapsed_time:.2f}s")

# Optionally, convert similarities_2019 into a DataFrame for easier analysis
similarities_df_2019 = pd.DataFrame(similarities_2019)

# You can now inspect the first few results
print(similarities_df_2019.head())


Processed speech 1/32874. Time elapsed: 0.81s
Processed speech 2/32874. Time elapsed: 1.47s
Processed speech 3/32874. Time elapsed: 2.27s
Processed speech 4/32874. Time elapsed: 2.95s
Processed speech 5/32874. Time elapsed: 3.66s
Processed speech 6/32874. Time elapsed: 4.33s
Processed speech 7/32874. Time elapsed: 5.00s
Processed speech 8/32874. Time elapsed: 5.68s
Processed speech 9/32874. Time elapsed: 6.35s
Processed speech 10/32874. Time elapsed: 7.01s
Processed speech 11/32874. Time elapsed: 7.66s
Processed speech 12/32874. Time elapsed: 8.33s
Processed speech 13/32874. Time elapsed: 9.01s
Processed speech 14/32874. Time elapsed: 9.67s
Processed speech 15/32874. Time elapsed: 10.33s
Processed speech 16/32874. Time elapsed: 10.98s
Processed speech 17/32874. Time elapsed: 11.78s
Processed speech 18/32874. Time elapsed: 12.44s
Processed speech 19/32874. Time elapsed: 13.11s
Processed speech 20/32874. Time elapsed: 13.79s
Processed speech 21/32874. Time elapsed: 14.45s
Processed speec

KeyboardInterrupt: 

In [25]:
import time

# Function to compute cosine similarity between two embeddings
def compute_cosine_similarity(embedding1, embedding2):
    return torch.mm(embedding1, embedding2.T).cpu().numpy()[0][0]

# Prepare an empty list to store the results
similarities_2019 = []

# Track the start time to estimate processing duration
start_time = time.time()

# Compute similarity for all speeches and news in 2019
for idx, speech_data in speeches_2019.iterrows():
    speech_embedding_2019 = speech_tensor_2019[idx]  # Use the index of the speech DataFrame to access tensor
    speech_date_2019 = speech_data['date']
    
    for news_id, news_data in news_2019.iterrows():
        news_embedding_2019 = news_tensor_2019[news_id]  # Use the index of the news DataFrame to access tensor
        news_date_2019 = news_data['Date']
        
        # Compute cosine similarity between the embeddings
        cosine_sim = compute_cosine_similarity(speech_embedding_2019, news_embedding_2019)
        
        # Append the result to the list
        similarities_2019.append({
            'speech_id': idx,  # Use the index from speeches_2019
            'news_id': news_id,
            'speech_date': speech_date_2019,
            'news_date': news_date_2019,
            'cosine_similarity': cosine_sim
        })
    
    # Track the elapsed time after processing each speech
    elapsed_time = time.time() - start_time
    print(f"Processed speech {idx + 1}/{len(speeches_2019)}. Time elapsed: {elapsed_time:.2f}s")


IndexError: index 116593 is out of bounds for dimension 0 with size 32874

# This is code for full dataset embeddings 

In [6]:
# Apply cleaning to the embeddings but store them in new variables
cleaned_news_embeddings = news_embeddings['news_embeddings'].apply(clean_and_convert)
cleaned_speech_embeddings = speeches_embeddings['speech_embeddings'].apply(clean_and_convert)

# Verify the cleaned embeddings
print("Cleaned news embedding (first sample):", cleaned_news_embeddings.iloc[0])
print("Cleaned speech embedding (first sample):", cleaned_speech_embeddings.iloc[0])

Cleaned news embedding (first sample): [ 3.77706587e-01  9.13102329e-02 -1.38176888e-01  4.42361206e-01
  3.78245354e-01  6.33672178e-02  6.13548398e-01 -4.14853036e-01
 -1.67487264e-01 -3.22665840e-01  6.13291383e-01 -1.93757042e-02
  3.53745461e-01  1.40970409e-01 -4.03713405e-01  3.24969411e-01
  4.17762280e-01 -1.92587554e-01  4.63847995e-01  2.11397439e-01
  8.83484781e-02  1.31108746e-01  3.07143807e-01 -4.08628702e-01
 -1.87494650e-01  5.12529425e-02  1.11430287e-02  4.37100828e-01
 -1.18361004e-01 -1.59158319e-01 -9.55921352e-01  6.38289630e-01
 -3.02048892e-01  5.60072422e-01 -2.31682688e-01  3.62528980e-01
  2.58382499e-01  1.92029942e-02  2.34295458e-01 -6.21018372e-02
 -1.33454457e-01 -6.48286581e-01  4.64166373e-01 -9.89553481e-02
  2.44500548e-01 -4.76573884e-01  1.52596474e-01  3.75475287e-02
 -1.19982101e-01  6.87674209e-02  4.93696570e-01 -8.35342169e-01
  1.05719477e-01 -4.44378793e-01 -5.53980827e-01  2.86331803e-01
 -6.79888964e-01 -7.89398886e-03 -7.73337632e-02 -5

In [7]:
import torch
import numpy as np

# Convert cleaned embeddings to NumPy arrays
news_matrix = np.vstack(cleaned_news_embeddings.values)
speech_matrix = np.vstack(cleaned_speech_embeddings.values)

# Convert NumPy arrays to PyTorch tensors and move to GPU
news_tensor = torch.tensor(news_matrix, dtype=torch.float32).cuda()
speech_tensor = torch.tensor(speech_matrix, dtype=torch.float32).cuda()


In [9]:
# Normalize the tensors along dimension 1 (row-wise)
news_tensor = news_tensor / news_tensor.norm(dim=1, keepdim=True)
speech_tensor = speech_tensor / speech_tensor.norm(dim=1, keepdim=True)


In [12]:
import numpy as np

# Path to save the similarity matrix in smaller chunks
chunk_size = 1000  # Adjust the chunk size based on available memory

# Save in smaller chunks
for i in range(0, cosine_sim_matrix.shape[0], chunk_size):
    chunk = cosine_sim_matrix[i:i + chunk_size]
    save_path = f'/kaggle/working/cosine_sim_matrix_chunk_{i // chunk_size}.npy'
    np.save(save_path, chunk)
    print(f"Chunk {i // chunk_size} saved to {save_path}")


OSError: [Errno 28] No space left on device

In [10]:
import time
import torch
import numpy as np

# Set a smaller chunk size to avoid memory overload
chunk_size = 500  # Start with smaller chunks if the kernel crashed previously
cosine_sim_matrix = np.zeros((speech_tensor.shape[0], news_tensor.shape[0]))  # Initialize result matrix

# Filepath to save progress
save_path = '/path/to/save/cosine_sim_matrix.npy'  # Change the path as needed

# Check if there's a previously saved progress
if os.path.exists(save_path):
    cosine_sim_matrix = np.load(save_path)
    print("Resumed from previous progress.")

# Record the start time of the full process
start_time = time.time()

# Process the speech tensor in smaller chunks
for i in range(0, speech_tensor.shape[0], chunk_size):
    # Get the current chunk of speech embeddings
    speech_chunk = speech_tensor[i:i + chunk_size]
    
    # Record the start time for this chunk
    chunk_start_time = time.time()

    # Ensure that no gradients are tracked
    with torch.no_grad():
        # Compute cosine similarity between this chunk of speech embeddings and all news embeddings
        sim_chunk = torch.mm(speech_chunk, news_tensor.T).cpu().numpy()  # Matrix multiplication
    
    # Store the results in the appropriate part of the cosine similarity matrix
    cosine_sim_matrix[i:i + chunk_size] = sim_chunk

    # Save progress after processing each chunk to avoid losing data
    np.save(save_path, cosine_sim_matrix)
    print(f"Progress saved to {save_path}")

    # Record the end time for this chunk
    chunk_end_time = time.time()

    # Print out the time taken for this chunk and the estimated time for all chunks
    chunk_time = chunk_end_time - chunk_start_time
    total_time = time.time() - start_time

    # Estimate the total time based on the processed chunks
    estimated_total_time = (total_time / (i + chunk_size)) * speech


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/save/cosine_sim_matrix.npy'

In [ ]:
import time
import torch

# Set a smaller chunk size to avoid memory overload
chunk_size = 500  # Start with smaller chunks if the kernel crashed previously
cosine_sim_matrix = np.zeros((speech_tensor.shape[0], news_tensor.shape[0]))  # Initialize result matrix

# Record the start time of the full process
start_time = time.time()

# Process the speech tensor in smaller chunks
for i in range(0, speech_tensor.shape[0], chunk_size):
    # Get the current chunk of speech embeddings
    speech_chunk = speech_tensor[i:i + chunk_size]
    
    # Record the start time for this chunk
    chunk_start_time = time.time()

    # Ensure that no gradients are tracked
    with torch.no_grad():
        # Compute cosine similarity between this chunk of speech embeddings and all news embeddings
        sim_chunk = torch.mm(speech_chunk, news_tensor.T).cpu().numpy()  # Matrix multiplication
    
    # Store the results in the appropriate part of the cosine similarity matrix
    cosine_sim_matrix[i:i + chunk_size] = sim_chunk

    # Record the end time for this chunk
    chunk_end_time = time.time()

    # Print out the time taken for this chunk and the estimated time for all chunks
    chunk_time = chunk_end_time - chunk_start_time
    total_time = time.time() - start_time

    # Estimate the total time based on the processed chunks
    estimated_total_time = (total_time / (i + chunk_size)) * speech_tensor.shape[0]

    print(f"Processed chunk {i + chunk_size}/{speech_tensor.shape[0]} - Time for this chunk: {chunk_time:.2f} seconds")
    print(f"Estimated total time: {estimated_total_time / 60:.2f} minutes")

    # Clear GPU memory after each chunk
    torch.cuda.empty_cache()

# Record the total time for the entire process
end_time = time.time()
total_process_time = end_time - start_time

# Print the total time for processing all chunks
print(f"\nTotal time to process all chunks: {total_process_time / 60:.2f} minutes")


In [ ]:
speech_ids = speeches_embeddings.index
news_ids = news_embeddings.index

# Convert the similarity matrix to a long-format DataFrame
similarity_df = pd.DataFrame(cosine_sim_matrix, index=speech_ids, columns=news_ids).reset_index().melt(id_vars='index', var_name='news_id', value_name='cosine_similarity')
similarity_df.rename(columns={'index': 'speech_id'}, inplace=True)

# Merge with date information if needed
similarity_df = similarity_df.merge(speeches_embeddings[['date']], left_on='speech_id', right_index=True)
similarity_df = similarity_df.merge(news_embeddings[['Date']], left_on='news_id', right_index=True)

# Inspect the result
print(similarity_df.head())


In [9]:
# Function to compute cosine similarity between two embeddings (CPU) 
def compute_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

# Prepare an empty list to store the results
similarities = []

# Compute similarity for all speeches and news
for speech_id, speech_data in speeches_embeddings.iterrows():
    speech_embedding = cleaned_speech_embeddings[speech_id]
    speech_date = speech_data['date']
    
    for news_id, news_data in news_embeddings.iterrows():
        news_embedding = cleaned_news_embeddings[news_id]
        news_date = news_data['Date']
        
        # Compute cosine similarity between the embeddings
        cosine_sim = compute_cosine_similarity(speech_embedding, news_embedding)
        
        # Append the result to the list
        similarities.append({
            'speech_id': speech_id,
            'news_id': news_id,
            'speech_date': speech_date,
            'news_date': news_date,
            'cosine_similarity': cosine_sim
        })

KeyboardInterrupt: 

In [ ]:
import time

# Sample size for estimation
sample_speeches = cleaned_speech_embeddings[:10]
sample_news = cleaned_news_embeddings[:10]

start_time = time.time()

for speech_embedding in sample_speeches:
    for news_embedding in sample_news:
        _ = compute_cosine_similarity(speech_embedding, news_embedding)

elapsed_time = time.time() - start_time
estimated_total_time = elapsed_time * (len(cleaned_speech_embeddings) / 10) * (len(cleaned_news_embeddings) / 10)

print(f"Estimated time for full computation: {estimated_total_time / 60:.2f} minutes")

In [66]:


# Convert the results to a DataFrame
similarity_df = pd.DataFrame(similarities)

# Inspect the results
print(similarity_df.head())


KeyboardInterrupt: 